In [10]:
import numpy as np
import pandas as pd
import pandasql as ps

In [118]:
countries = ['China', 'Germany', 'India', 'United States']

df = pd.read_csv('./Data/policies.csv', index_col=0)
df = df[df['Year'] >= 2000]
df = df.replace('People\'s Republic Of China', countries[0])

In [119]:
def process_sub_df(df, status):
    start = 2000
    end = 2017
    size = end - start
    
    p_sub_df_year = np.array([])
    p_sub_df_country = np.array([])
    p_sub_df_policies = np.array([])

    df = df[df['Status'] == status].groupby(['Country', 'Year']).count()
    df = df.reset_index(level=[0, 1])
    
    for i, country in enumerate(countries):
        idx = np.arange(start, end)
        
        c_df = df[df['Country'] == country]
        c_df.index = c_df['Year']
        c_df = c_df.drop(['Country', 'Year'], axis=1)
        c_df = c_df.reindex(idx, fill_value=0)
        c_df = c_df.reset_index()
        
        p_sub_df_year = np.append(p_sub_df_year, c_df['Year'])
        p_sub_df_country = np.append(p_sub_df_country, [country] * (end - start))
        p_sub_df_policies = np.append(p_sub_df_policies, c_df['Policy'])
        
    processed_sub_df = pd.DataFrame({'Year': p_sub_df_year, 
                                     'Country': p_sub_df_country,
                                     'Policies': p_sub_df_policies})
    return processed_sub_df

in_force = process_sub_df(df, 'In force')
ended = process_sub_df(df, 'Ended')

In [125]:
query = """
    SELECT Year, Country, COUNT(*) AS Policies
    FROM in_force
    WHERE Year >= 2000
    GROUP BY Year, Country
    ORDER BY Country
"""

in_force = ps.sqldf(query)
in_force['Policies'] = in_force.groupby('Country')['Policies'].cumsum()

In [133]:
df = in_force.merge(ended, on=['Country', 'Year'])
df['Number of policies'] = df['Policies_x'] + df['Policies_y']
df = df.drop(['Policies_x', 'Policies_y'], axis=1)

In [137]:
df.to_csv('./Data Formated/policies.csv')